In [2]:
// SETUP
use std::fs::File;
use std::collections::{HashMap, HashSet};
use std::str::FromStr;
use std::io::{BufRead, BufReader, Error, ErrorKind, Read};

fn read_lines(path: &str) -> Result<Vec<String>, Error> {
    let file = File::open(path)?; 
    BufReader::new(file).lines().collect()
}

fn read_integers(path: &str) -> Result<Vec<i64>, Error> {
    let mut v = Vec::new();
    for line in read_lines(path)? {
        let n = line   
            .trim() 
            .parse() 
            .map_err(|e| Error::new(ErrorKind::InvalidData, e))?; 
        v.push(n);
    }
    Ok(v)
}

const DATA_ROOT: &str = "data/";

fn data_file(problem_id: u32) -> String {
    format!("{}/{:02}.txt", DATA_ROOT, problem_id)
}

In [65]:
// DAY01
const TOTAL: i64 = 2020;

fn problem01() {    
    let nums: Vec<i64> = read_integers(&data_file(1)).unwrap();    
    let mut reg = HashMap::new();

    for (i, n) in nums.iter().enumerate() {
        let rest = TOTAL - n;
        if reg.contains_key(&rest) {
            println!("{} * {} = {}", n, rest, n * rest);
        }
        reg.insert(n, i);
    }
    
    for (i, n1) in nums.iter().enumerate() {
        for j in (i + 1)..nums.len() {
            let n2 = nums[j];
            let rest = TOTAL - n1 - n2;
            match reg.get(&rest) {
                Some(k) if k > &j => {
                    println!("{} * {} * {} = {}", n1, n2, rest, n1 * n2 * rest)
                },
                _ => (),
            }
        }
    }
}

problem01();

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [50]:
// DAY02
#[derive(Debug, PartialEq)]
struct PasswordDesc {
    cnt1: usize,
    cnt2: usize,
    ch: char,
    pwd: String,
}

impl FromStr for PasswordDesc {
    type Err = std::num::ParseIntError;
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let parts: Vec<&str> = s.split(' ').collect();
        let counts: Vec<usize> = parts[0]
            .split('-')
            .map(|p| p.parse().unwrap())
            .collect();
        Ok(PasswordDesc { 
            cnt1: counts[0], 
            cnt2: counts[1], 
            ch: parts[1].as_bytes()[0] as char, 
            pwd: parts[2].to_owned()})
    }
}

fn is_valid1(p: &PasswordDesc) -> bool {
    let cnt = p.pwd.matches(p.ch).count();
    p.cnt1 <= cnt && cnt <= p.cnt2
}

fn is_valid2(p: &PasswordDesc) -> bool {
    let pwd = p.pwd.as_bytes();
    let ch = p.ch as u8;
    (pwd[p.cnt1 - 1] == ch) ^ (pwd[p.cnt2 - 1] == ch)
}

fn problem02() {
    let passwords: Vec<PasswordDesc> = 
        read_lines(&data_file(2)).unwrap()
        .iter()
        .filter_map(|line| line.parse().ok())
        .collect();

    println!("Number of valid passwords: {}, {}", 
        passwords.iter().filter(|x| is_valid1(x)).count(), 
        passwords.iter().filter(|x| is_valid2(x)).count());
}

problem02();

Number of valid passwords: 548, 502


In [72]:
// DAY03
const TREE: u8 = '#' as u8;

fn count_trees(rows: &Vec<String>, dx: usize, dy: usize) -> usize {
    let h = rows.len();
    if h == 0 {
        return 0;
    }
    let w = rows[0].len();
    let mut x: usize = 0;
    let mut res: usize = 0;
    for y in (0..h).step_by(dy) {
        res += (rows[y].as_bytes()[x % w] == TREE) as usize;
        x += dx;
    }
    res
}

fn problem03() {
    let rows = read_lines(&data_file(3)).unwrap();
    let res1 = count_trees(&rows, 3, 1);
    println!("Num trees 1: {}", res1);
    
    let res2: usize = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
        .iter()
        .map(|(dx, dy)| count_trees(&rows, *dx, *dy))
        .product();
    println!("Num trees 2: {}", res2);
}

problem03();

Num trees 1: 242
Num trees 2: 2265549792


In [93]:
// DAY04
#[macro_use] extern crate lazy_static;
extern crate regex;

use regex::Regex;
use std::fs;
use std::collections::HashSet;

type Passport = HashMap<String, String>;

fn read_passport(line: &str) -> Passport {
    line.split_whitespace()
        .filter_map(|s| {
            let mut parts = s.split(':');
            Some((parts.next()?.to_owned(), parts.next()?.to_owned())) 
        })
        .collect()
}

const REQUIRED_FIELDS: [&str; 7] = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"];

fn is_valid1(passport: &Passport) -> bool {
    REQUIRED_FIELDS.iter().all(|f| passport.contains_key(*f))    
}

fn is_valid2(passport: &Passport) -> bool {
    lazy_static! {
        static ref PID_REGEX: Regex = Regex::new(r"^(\d{9})$").unwrap();
        static ref HCL_REGEX: Regex = Regex::new(r"^#[0-9a-f]{6}$").unwrap();
        static ref HGT_REGEX: Regex = Regex::new(r"^(?P<in>\d+)in|(?P<cm>\d+)cm$").unwrap();
        static ref EYE_COLORS: HashSet<String> = ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
            .iter()
            .copied()
            .map(|x| x.to_owned())
            .collect();
    }
    
    fn in_range<T: Ord + FromStr>(int_str: &str, minv: T, maxv: T) -> bool {
        match int_str.parse::<T>() {
            Ok(n) => n >= minv && n <= maxv,
            _ => false
        }
    }
    
    fn is_valid_height(height_str: &str) -> bool {
        match HGT_REGEX.captures(height_str) {
            Some(c) => 
                match (c.name("in"), c.name("cm")) {
                    (Some(x), _) => in_range(x.as_str(), 59, 76),
                    (_, Some(x)) => in_range(x.as_str(), 150, 193),
                    _ => false,
                },
            _ => false
        }
    }
    
    is_valid1(passport) &&
    in_range(&passport["byr"], 1920, 2002) &&
    in_range(&passport["iyr"], 2010, 2020) &&
    in_range(&passport["eyr"], 2020, 2030) &&
    EYE_COLORS.contains(passport["ecl"].as_str()) &&
    PID_REGEX.is_match(&passport["pid"]) &&
    HCL_REGEX.is_match(&passport["hcl"]) &&
    is_valid_height(&passport["hgt"])
}

fn problem04() {
    let passports: Vec<Passport> = fs::read_to_string(&data_file(4))
        .unwrap()
        .split("\n\n")
        .map(|s| read_passport(s))
        .collect();

    println!("Valid passports 1: {}", passports.iter().filter(|x| is_valid1(x)).count());
    println!("Valid passports 2: {}", passports.iter().filter(|x| is_valid2(x)).count());
}

problem04();

Valid passports 1: 264
Valid passports 2: 224


In [50]:
// DAY05
use std::collections::HashSet;
use std::iter::FromIterator;

fn get_id(seat: &str) -> u32 {
    seat.chars().fold(0, |res, c| res * 2 + "BR".contains(c) as u32)
}

fn problem05() {
    let lines = read_lines(&data_file(5)).unwrap();
    let ids: Vec<u32> = lines
        .iter()
        .map(|x| get_id(x))
        .collect();
    
    let max_id: u32 = *ids.iter().max().unwrap();
    let id_set = ids.iter().cloned().collect::<HashSet<u32>>();
    
    let mut seat: u32 = 0;
    for i in 0..=(1 << lines[0].len()) {
        let id = i as u32;
        if !id_set.contains(&id) && 
            id_set.contains(&(id + 1)) && 
            id_set.contains(&(id - 1)) {
            seat = id;
            break;
        }
    }
    println!("Max ID: {}, Seat: {}", max_id, seat);
}

problem05();

Max ID: 801, Seat: 597


In [44]:
// DAY06
use std::fs;
use std::collections::HashMap;

fn problem06() {
    let text = fs::read_to_string(&data_file(6)).unwrap();
    let groups: Vec<Vec<&str>> = text
            .split("\n\n")
            .map(|s| s.split_whitespace().collect())
            .collect();
    
    let (mut res1, mut res2) = (0, 0);
    for group in &groups {
        let mut counts: HashMap<char, usize> = HashMap::new();
        for answers in group {
            for ans in answers.chars() {
                *counts.entry(ans).or_insert(0) += 1;
            }
        }
        res1 += counts.len();
        res2 += counts.iter().filter(|&(_, v)| *v == group.len()).count()
    }
    println!("Answer 1: {}, Answer 2: {}", res1, res2);
}

problem06();

Answer 1: 6310, Answer 2: 3193


In [63]:
// DAY07
type Mapping = HashMap<String, HashMap<String, u32>>;
type BackMapping = HashMap<String, Vec<String>> ;

fn parse_mapping(line: &str) -> (String, HashMap<String, u32>) {
    let parts: Vec<&str> = line.split_whitespace().collect();
    let bag_color = parts[0..=1].join(" ");
    let mut res: HashMap<String, u32> = HashMap::new();
    for i in (4..parts.len()).step_by(4) {
        if parts[i] != "no" {
            let child_bag_color = parts[i + 1..i + 3].join(" ");
            res.insert(child_bag_color, parts[i].parse::<u32>().unwrap());
        }
    }
    (bag_color, res)
}

fn build_back_mapping(mapping: &Mapping) -> BackMapping {
    let mut res: HashMap<_, _> = HashMap::new();
    for (key, children) in mapping {
        for (child, _) in children {
            res.entry(child.clone()).or_insert(Vec::new()).push(key.clone());
        }
    }
    res
}

fn count_reachable(root: &str, back_mapping: &BackMapping) -> u32 {
    fn rec(node: &str, back_mapping: &BackMapping, visited: &mut HashSet<String>) -> u32 {
        if visited.contains(node) {
            return 0;
        }
        let mut res = 1;
        if back_mapping.contains_key(node) {
            for child in &back_mapping[node] {
                res += rec(&child, back_mapping, visited);
            }
        }
        visited.insert(node.to_owned());
        res
    }
    let mut visited: HashSet<String> = HashSet::new();
    rec(root, back_mapping, &mut visited)
}

fn count_contains(root: &str, mapping: &Mapping) -> u32 {
    let mut res: u32 = 1;
    for (child, count) in &mapping[root] {
        res += count * count_contains(&child, mapping);
    }
    res
}

fn problem07() {
    let mapping: Mapping = read_lines(&data_file(7))
        .unwrap()
        .iter()
        .map(|line| parse_mapping(line))
        .into_iter()
        .collect();
    let back_mapping = build_back_mapping(&mapping);
    const ROOT: &str = "shiny gold";
    println!("Reachable: {}", count_reachable(ROOT, &back_mapping) - 1);
    println!("Contains: {}", count_contains(ROOT, &mapping) - 1);
}

problem07();

Reachable: 370
Contains: 29547


In [42]:
// DAY08

#[derive(Debug, PartialEq, Copy, Clone)]
enum Op {
    Acc(i64),
    Nop(i64),
    Jmp(i64),
};

impl FromStr for Op {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let mut it = s.split(' ');
        let opstr: &str = it.next().ok_or(())?;
        let val: i64 = it.next().ok_or(())?.parse::<i64>().map_err(|_|())?;
        match opstr {
            "acc"  => Ok(Op::Acc(val)),
            "nop"  => Ok(Op::Nop(val)),
            "jmp"  => Ok(Op::Jmp(val)),
            _      => Err(()),
        }
    }
}

fn eval_ops(ops: &Vec<Op>) -> (bool, i64) {
    let n = ops.len() as i64;
    let mut visited: Vec<bool> = Vec::new();
    visited.resize(ops.len(), false);
    let mut ip: i64 = 0;
    let mut acc: i64 = 0;
    while ip >= 0 && ip < n {
        if visited[ip as usize] {
            return (false, acc)
        }
        visited[ip as usize] = true;
        match ops[ip as usize] {
            Op::Jmp(val) => ip += val - 1,
            Op::Acc(val) => acc += val,
            _ => (),
        }
        ip += 1;
    }
    (true, acc)
}

fn try_mutate_program(ops: &Vec<Op>) -> Option<i64> {
    let mut ops_copy = ops.clone();
    for (i, op) in ops.iter().enumerate() {
        let prevOp: Op = *op;
        let newOp = match op {
            Op::Jmp(val) => Op::Nop(*val),
            Op::Nop(val) => Op::Jmp(*val),
            _ => prevOp
        };
        if prevOp != newOp {
            ops_copy[i] = newOp;
            let (terminated, acc) = eval_ops(&ops_copy);
            if terminated {
                return Some(acc);
            }
            ops_copy[i] = prevOp;
        }
    }
    None
}

fn problem08() {
    let ops: Vec<Op> = 
        read_lines(&data_file(8)).unwrap()
        .iter()
        .filter_map(|line| line.parse().ok())
        .collect();
    let (_, acc) = eval_ops(&ops);
    println!("Answer 1: {}", acc);
    println!("Answer 2: {}", try_mutate_program(&ops).unwrap());
}

problem08();

Answer 1: 1782
Answer 2: 797
